In [2]:
"""
This script preps the reference and testing images for the agreement assessment

    - Resamples existing products to common 10m resolution matching the Sentinel-based aspen map

LANDFIRE Existing Vegetation Type (EVT, c. 2016 Remap)
USFS National Individual Tree Species Atlas (c. 2014)
USFS TreeMap (c. 2016)

maxwell.cook@colorado.edu
"""

# Packages

import os,sys
import geopandas as gpd
import rioxarray as rxr
import rasterio
from rasterio.enums import Resampling

import time
begin = time.time()

# Custom functions
# Function to resample and snap grids
def resample_match_grid(
        in_img, to_img, scale_factor,
        crs, dtype, method='', out_path=os.getcwd()):

    # Define the resample dimensions
    new_height = int(in_img.rio.height * scale_factor)
    new_width = int(in_img.rio.width * scale_factor)

    # Reproject w/ resampling
    resamp = in_img.rio.reproject(
        crs,
        shape=(new_height, new_width),
        resampling=method
    )

    del in_img

    # Reproject match, clip, save out
    out_grid = resamp.rio.reproject_match(to_img)

    del to_img, resamp

    out_grid.rio.to_raster(
        out_path, compress='zstd', zstd_level=9,
        dtype=dtype, driver='GTiff'
    )

    del out_grid

# Globals

proj = 'EPSG:5070'  # NAD83 CONUS Albers

# Load the study area boundaries
# Southern Rocky Mountains Ecoregion (SRME)
srme = gpd.read_file('us_eco_l3_srme.gpkg').to_crs(proj)
# White River National Forest
wrnf = gpd.read_file('wrnf_boundary_srme.gpkg').to_crs(proj)
# Combine in a list
gdfs = [srme,wrnf]
names = ["srme","wrnf"]

#############################
# Prep the reference images #
#############################

print("Starting prep of reference images ...")

# Target grid (Sentinel-based aspen distribution map)
tests = [
    's2aspen_prob_10m_binOpt_srme.tif',
    's2aspen_prob_10m_binOpt_wrnf.tif'
]

# Load the "reference" images of aspen distribution
# USFS TreeMap, ITSP (basal area), LANDFIRE EVT
refs = [
    'lc16_evt_200_bin.tif',
    'usfs_itsp_aspen_ba_gt10.tif',
    'usfs_treemap16_bin.tif'
]

# Loop the reference images, exporting a matched 10-meter grid

refs = ['usfs_treemap16_bin.tif']

for r in refs:
    name = os.path.basename(r)[:-4]
    print(f"Processing: {name}")

    # Open the raster file
    ref = rxr.open_rasterio(r,masked=True,cache=False).squeeze().astype(rasterio.uint8)

    for i in range(0,len(names)):
        print(f"Clipping to the '{names[i]}'")

        gdf = gdfs[i]
        region = names[i]

        # Open the test image to match
        test = rxr.open_rasterio(tests[i],masked=True,cache=False).astype(rasterio.uint8)

        # Clip to study region
        cl = ref.rio.clip(gdf.geometry)

        # Upscale to 10m and match to the test image
        print("Upsampling to 10m")

        out = f'{name}_{region}_10m.tif'

        ups = resample_match_grid(
            in_img=cl, to_img=test,
            scale_factor=3, crs=proj, method=Resampling.nearest,
            dtype="uint8", out_path=out
        )

        del test, cl, ups

    del ref

print(f"Time elapsed: {(time.time() - begin) / 60} minutes.")

Starting prep of reference images ...
Processing: usfs_treemap16_bin
Clipping to the 'srme'


/opt/conda/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/opt/conda/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


Upsampling to 10m
Clipping to the 'wrnf'


/opt/conda/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/opt/conda/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:188: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


Upsampling to 10m
Time elapsed: 5.914511028925578 minutes.
